# Position Reconstruction for XeBRA - Comparison

**Status:** August 2019

Position reconstruction for XeBRA using Centroids, LRFs and NNs as well as comparison performance simulations vs measurements.

In [1]:
## Imports:

import sys
import glob

import numpy as np
from numpy import exp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
from matplotlib.image import NonUniformImage
from matplotlib.patches import Rectangle
%matplotlib inline

import pandas as pd
import math

from scipy import optimize
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy import stats
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.interpolate import spline
from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import griddata
from scipy.stats import chisquare
from scipy.stats import power_divergence

## General Definitions and Parameters

In [2]:
## General function definitions:

def f_polar_R(cartesian_x, cartesian_y):
    return np.sqrt(cartesian_x * cartesian_x + cartesian_y * cartesian_y)

def f_polar_Phi(cartesian_x, cartesian_y):
    return np.arctan2(cartesian_y, cartesian_x)

def f_distance(x_a, y_a, x_b, y_b):
    return np.sqrt((x_b - x_a)**2 + (y_b - y_a)**2)


def flatten_tuples(tup):
    return np.array([element for tupl in tup for element in tupl])

def sum_tuple_subentries(tup):
    return [np.sum(tupl) for tupl in tup]

def nth_tuple_subentries(tup, n):
    return [tupl[n] for tupl in tup]


def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [3]:
## PMT properties
R8520_body_width = 25.7
R8520_body_cornerradius = 2.
R8520_photocathode_width = 20.5
R8520_photocathode_cornerradius = 1.
PMT_ID = np.array([1,2,3,4,5,6,7])
PMT_position_x = np.array([-14.,-28,-14.,14.,28.,14.,0.])      # x-position PMTs in mm in cartesian coordinates
PMT_position_y = np.array([-28.,0.,28.,28.,0.,-28.,0.])        # y-position PMTs in mm in cartesian coordinates
PMT_position_R = f_polar_R(PMT_position_x, PMT_position_y)     # radial position PMTs in mm in polar coordinates
PMT_position_Phi = f_polar_Phi(PMT_position_x, PMT_position_y) # angular position PMTs in rad in polar coordinates
PMT_positions = pd.DataFrame(index=PMT_ID, data={'PMT_ID': PMT_ID, 'PMT_position_x': PMT_position_x, 'PMT_position_y': PMT_position_y, 'PMT_position_R': PMT_position_R, 'PMT_position_Phi': PMT_position_Phi})

## Imports and Data Processing

### Measurements

In [5]:
!ls /home/ab602/Thesis/Xebra_Measurements_Analysis/Position_Reconstruction_*/output/run_00158*.pkl
!ls /home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_*/output/run_00158*.pkl

ls: cannot access '/home/ab602/Thesis/Xebra_Measurements_Analysis/Position_Reconstruction_*/output/run_00158*.pkl': No such file or directory
/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_Centroid/output/run_00158_Kr83m_S2_AFT_Reconstruction_Centroid.pkl
/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_LRF/output/run_00158_Kr83m_S2_AFT_Iterative_Reconstruciton.pkl
/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_NN/output/run_00158_Kr83m_S2_AFT_Reconstruction_NN.pkl


In [9]:
#### Import processed measurements data

## Define run to be used
## Good Kr83m runs concerning S2 AFT: 00157, 00158, (00159,) 00163; best: 00158
run = "00158"

## Import pandas dataframe from pkl
try:
    df_Centroid = pd.read_pickle("/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_Centroid/output/run_"+run+"_Kr83m_S2_AFT_Reconstruction_Centroid.pkl")
    df_LRF      = pd.read_pickle("/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_LRF/output/run_"+run+"_Kr83m_S2_AFT_Iterative_Reconstruciton.pkl")
    df_NN       = pd.read_pickle("/home/alex/Thesis-Copy/Xebra_Measurements_Analysis/Position_Reconstruction_NN/output/run_"+run+"_Kr83m_S2_AFT_Reconstruction_NN.pkl")
    file_exist = True
except:
    pass
try:
    df_Centroid = pd.read_pickle("/home/ab602/Thesis/Xebra_Measurements_Analysis/Position_Reconstruction_Centroid/output/run_"+run+"_Kr83m_S2_AFT_Reconstruction_Centroid.pkl")
    df_LRF      = pd.read_pickle("/home/ab602/Thesis/Xebra_Measurements_Analysis/Position_Reconstruction_LRF/output/run_"+run+"_Kr83m_S2_AFT_Iterative_Reconstruciton.pkl")
    df_NN       = pd.read_pickle("/home/ab602/Thesis/Xebra_Measurements_Analysis/Position_Reconstruction_NN/output/run_"+run+"_Kr83m_S2_AFT_Reconstruction_NN.pkl")
except:
    if file_exist:
        pass
    else:
        print('Selected file cannot be found.')
    
filenameprefix_meas = "run_"+run+"_Kr83m_S2_AFT"
print(filenameprefix_meas)

run_00158_Kr83m_S2_AFT


In [21]:
x_pos_Centroid = np.array([df_Centroid['x_pos']])
y_pos_Centroid = np.array([df_Centroid['y_pos']])

df_Centroid.head()

,x_pos,y_pos
0,10.786233,10.913062
1,13.446973,12.508565
2,-13.775311,-3.993569
3,1.341471,-4.348050
4,-13.210608,-1.859828


In [22]:
x_pos_LRF_iteration0 = df_LRF['x_pos']['0']
x_pos_LRF_iteration1 = df_LRF['x_pos']['1']
x_pos_LRF_iteration2 = df_LRF['x_pos']['2']
x_pos_LRF_iteration3 = df_LRF['x_pos']['3']
x_pos_LRF_iteration4 = df_LRF['x_pos']['4']
x_pos_LRF_iteration5 = df_LRF['x_pos']['5']
y_pos_LRF_iteration0 = df_LRF['y_pos']['0']
y_pos_LRF_iteration1 = df_LRF['y_pos']['1']
y_pos_LRF_iteration2 = df_LRF['y_pos']['2']
y_pos_LRF_iteration3 = df_LRF['y_pos']['3']
y_pos_LRF_iteration4 = df_LRF['y_pos']['4']
y_pos_LRF_iteration5 = df_LRF['y_pos']['5']

df_LRF.head()

,rfiducial,rfiducial_under,rfiducial_central,fit_parameter,x_pos,y_pos,r_pos_binned,HF_binned,fit_parameter_end
0,20,15,32,"[[0.5862075, 30.03753042, -0.34682365, 4.18897...","[15.067650109682376, 23.21576116642172, -19.06...","[15.013278202566628, 18.183364395947017, -6.74...","[[1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5,...","[[0.6148002732025396, 0.6026768041361014, 0.59...","[[0.6016925420036606, 36.061586747735234, -3.0..."
1,20,15,32,"[[0.6016925420036606, 36.061586747735234, -3.0...","[14.877927176548072, 21.528979814798372, -18.9...","[14.669509095254746, 17.167255646907062, -6.68...","[[2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5...","[[0.6285648450994098, 0.6083875843289961, 0.59...","[[0.6213197917541504, 36.69817529257881, -3.23..."
2,20,15,32,"[[0.6213197917541504, 36.69817529257881, -3.23...","[14.475328762864487, 21.130269711541644, -18.7...","[14.392954018545405, 16.919138998592572, -6.53...","[[3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11....","[[0.6201795732498544, 0.6042094375186153, 0.58...","[[0.6262110565403661, 35.62948794775529, -2.63..."
3,20,15,32,"[[0.6262110565403661, 35.62948794775529, -2.63...","[14.200224175195073, 20.65097662243528, -18.50...","[14.133531136213001, 16.592993009023054, -6.40...","[[4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12...","[[0.6113781291003003, 0.5969387524238264, 0.57...","[[0.6296958552450105, 33.84151139852634, -1.86..."
4,20,15,32,"[[0.6296958552450105, 33.84151139852634, -1.86...","[13.922488346083757, 19.703058890654994, -18.2...","[13.876226052984217, 16.0275835695048, -6.2783...","[[4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12...","[[0.6151910109773345, 0.6013939293035435, 0.58...","[[0.6342629093210318, 34.36383729277455, -2.05..."


In [23]:
x_pos_NN = np.array([df_NN['x_pos']])
y_pos_NN = np.array([df_NN['y_pos']])

df_NN.head()

,x_pos,y_pos
0,15.291557,15.212932
1,24.357395,18.508663
2,-19.461060,-6.906969
3,2.264139,-7.366965
4,-18.602991,-3.554627


In [28]:
x_pos_NN.shape

(1, 18827)

### Simulations

In [24]:
df_NN_sim = pd.read_pickle("/media/alex/TOSHIBA EXT/Simulation_Outputs/Position_Reconstruction_NN/MCfakemeasurement_optPhot_S2_1.0e+06_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_Reconstruction_NN.pkl")

In [25]:
df_NN_sim

,x_pos,y_pos
0,18.397106,17.579727
1,31.665052,-2.186015
2,-16.702953,22.105122
3,-25.734169,16.920139
4,18.514349,-11.042530
5,16.002428,-1.767958
6,-3.453744,9.921898
7,29.494720,9.028007
8,-3.272428,29.475292
9,-1.336394,-3.713645


In [ ]:
"/media/alex/TOSHIBA EXT/Simulation_Outputs/Position_Reconstruction_NN/MCfakemeasurement_optPhot_S2_1.0e+06_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_Reconstruction_NN.pkl"

scale by difference statistics